<center><h1> numerical result of "solar + hold-out" </h1></center>

---

## Check the following before running the code

### (a) Read "Read_Me_First.docx" first, which introduces the package

### (b) Before replication, delete all .p files in the "./numerical_result" folder. The .p files record the numerical results of the our computation.

### (c) To avoid confusion, reset your kernel before you running the notebook (to clear memory): Menu "Kernel" $\rightarrow$ "Restart Kernel and clear all outputs".

### (d) To evaluate the code for simulation replication,
* <font size="4.5"> click : Menu "Kernel" $\rightarrow$ "Restart Kernel and Run All Cells" </font>
* <font size="4.5"> or, select a cell of code, press "shift" and "enter". Run all cells to avoid errors </font>

### (e) Check "joblib", "scikit-learn", "numpy", "matplotlib" and "tqdm" are installed. If not,
* <font size="4.5"> run "pip install joblib scikit-learn numpy matplotlib tqdm" in terminal (Mac OS or Linux) or command prompt as administrator (Windows) if you use Python3 without any IDE. </font>
* <font size="4.5"> we highly recommend installing Anaconda3 version 2020-11 directly to avoid package management (all packages mentioned above are installed by default).</font>

---

## #1: import all modules

* <font size="4.5"> "pickle" is used to save all computation results into ".p" files, which can be loaded later. </font>

* <font size="4.5"> For simplicity and elegancy, all relevant functions and classes are coded in "simul_plot_holdout_average_parallel.py". </font>

In [1]:
%reset -f

from simul_plot_holdout_average_parallel import simul_plot

import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import pickle
import os
import errno

---

## #2: define all functions

## #2(a): computation and visualization

In [2]:
def func_simul(sample_size, n_dim, n_info, num_rep, step_size, rnd_seed, repro):
    
    #set random seed
    np.random.seed(rnd_seed)

    print("compute the simulation with sample size "+str(sample_size)+" and number of variables "+str(n_dim))

    trial = simul_plot(sample_size, n_dim, n_info, num_rep, step_size, rnd_seed)

    pkl_file = "./numerical_result/solar_simul_n_" + str(sample_size) + "_p_" + str(n_dim) + "_hold_out.p"

    if repro == True:

        _, solar_Q_opt_c_stack = trial.simul_func()

        #save all the computation result into Pickle files
        #create the subdirectory if not existing
        if not os.path.exists(os.path.dirname(pkl_file)):
            try:
                os.makedirs(os.path.dirname(pkl_file))
            # Guard against race condition
            except OSError as exc: 
                if exc.errno != errno.EEXIST:
                    raise
        with open(pkl_file, "wb") as f:
            pickle.dump( solar_Q_opt_c_stack, f)

    else:
        with open(pkl_file, "rb") as f:
            solar_Q_opt_c_stack = pickle.load( f )

    #compute the number of selected variable of solar
    ## set the container
    solar_len_array = np.empty([len(solar_Q_opt_c_stack)])

    ##count the number 
    for i in range(num_rep):
        solar_len_array[i]  = len(solar_Q_opt_c_stack[i])

    #compute the marginal probability of selecting each informative variable
    ## set the container
    solar_var_array  = np.empty([5])

    ##concatenate results
    solar_vari_appe_stack  = np.concatenate(solar_Q_opt_c_stack ,0)

    ##count the number 
    for i in range(5):
        solar_var_array[i]  = (solar_vari_appe_stack  == i).sum()/num_rep

    #sparsity table
    mean_col   = [np.mean(solar_len_array)]
    median_col = [np.median(solar_len_array)]
    IQR_col    = [np.quantile(solar_len_array , 0.75) - np.quantile(solar_len_array , 0.25)]

    df1 = pd.concat([pd.DataFrame({'algo':['solar + hold-out']}), 
                     pd.DataFrame({'Number of selected variables (mean)': mean_col}),
                     pd.DataFrame({'Number of selected variables (median)': median_col}),
                     pd.DataFrame({'Number of selected variables (IQR)': IQR_col})], 
                    axis=1, join='inner')

    #accuracy table
    prob_0 = [solar_var_array[0]]
    prob_1 = [solar_var_array[1]]
    prob_2 = [solar_var_array[2]]
    prob_3 = [solar_var_array[3]]
    prob_4 = [solar_var_array[4]]

    df2 = pd.concat([pd.DataFrame({'algo':['solar + hold-out']}), 
                     pd.DataFrame({'Pr(select X1)': prob_0}),
                     pd.DataFrame({'Pr(select X1)': prob_1}),
                     pd.DataFrame({'Pr(select X1)': prob_2}),
                     pd.DataFrame({'Pr(select X1)': prob_3}),
                     pd.DataFrame({'Pr(select X1)': prob_4})],
                    axis=1, join='inner')

    return df1.round(2), df2.round(2)

---

## #3(a): define inputs values

| <font size="4.5"> variable name </font> | <font size="4.5">  meaning </font> |
|-|-|
| <font size="4.5">  sample_size  </font> | <font size="4.5">  the sample size $n$ in the paper; </font>| 
| <font size="4.5">  n_dim        </font> | <font size="4.5">  the number of variables (informative + redundant) in $X$, $p$ in the paper; </font>| 
| <font size="4.5">  n_info       </font> | <font size="4.5">  the number of informative variables in $X$; </font>| 
| <font size="4.5">  n_repeat     </font> | <font size="4.5">  the number of subsamples generated by solar; </font>| 
| <font size="4.5">  num_rep      </font> | <font size="4.5">  the total repetition number of this simulation; </font>|
| <font size="4.5">  step_size    </font> | <font size="4.5">  the step size for tuning $c$; </font>| 
| <font size="4.5">  rnd_seed     </font> | <font size="4.5">  the random seed value; </font>| 

### #3(b): define DGP

* <font size="4.5"> the population regression equation is $$Y = 2\cdot \mathbf{x}_0 + 3\cdot \mathbf{x}_1 + 4\cdot \mathbf{x}_2 + 5\cdot \mathbf{x}_3 + 6\cdot \mathbf{x}_4  + u,$$ 
* <font size="4.5"> To change the simulation settings, simply change the input values. If you change *n_info* you will adjust the DGP as follows: </font>
    * <font size="4.5"> If $i > \mbox{n_info} - 1$ and $i \in \left[ 0, 1, 2, \ldots, p-1 \right]$, $\beta_i = 0$ in population;</font>
    * <font size="4.5"> If $i \leqslant \mbox{n_info} - 1$ and $i \in \left[ 0, 1, 2, \ldots, p-1 \right]$, $\beta_i = i + 2$ in population</font>

In [3]:
n_dim     = 100
n_info    = 5
step_size = -0.01
num_rep   = 200
rnd_seed  = 0

sample_size_0 = 100
sample_size_1 = 150
sample_size_2 = 200

---

## #4: Result

 * <font size="4.5"> if you only want to see the raw result, please leave variable *repro* as boolean *False* </font>
 * <font size="4.5"> for replication, please change variable *repro* to boolean *True* </font>

In [4]:
repro = False

  * ### Numpy, Sklearn and Python are actively updated. If you use different version, replication results may be slightly different from the paper (see Read_me_first.pdf for detail).

  * ### To rerun this part, first delete all .p files in the "raw_result" folder to avoid possible bug.

# Part 1 : $p/n \rightarrow 0$

## #4(a): $p/n=100/100$ 

In [5]:
df1, df2 = func_simul(sample_size_0, n_dim, n_info, num_rep, step_size, rnd_seed, repro)

  0%|          | 0/200 [00:00<?, ?it/s]

compute the simulation with sample size 100 and number of variables 100


100%|██████████| 200/200 [00:19<00:00, 10.31it/s]


In [6]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,5.02,5.0,0.0


In [7]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,0.99,0.99,0.99,0.99,0.99


## #4(b): $p/n=100/150$

In [8]:
df1, df2 = func_simul(sample_size_1, n_dim, n_info, num_rep, step_size, rnd_seed, repro)

  0%|          | 1/200 [00:00<00:20,  9.68it/s]

compute the simulation with sample size 150 and number of variables 100


100%|██████████| 200/200 [00:19<00:00, 10.15it/s]


In [9]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,5.12,5.0,0.0


In [10]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,1.0,1.0,1.0,1.0,1.0


## #4(c): $p/n=100/200$

In [11]:
df1, df2 = func_simul(sample_size_2, n_dim, n_info, num_rep, step_size, rnd_seed, repro)

  0%|          | 1/200 [00:00<00:21,  9.40it/s]

compute the simulation with sample size 200 and number of variables 100


100%|██████████| 200/200 [00:20<00:00,  9.71it/s]


In [12]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,5.17,5.0,0.0


In [13]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,1.0,1.0,1.0,1.0,1.0


# Part 2 : $p/n \rightarrow 1$

In [14]:
sample_size_3 = 100; n_dim_3 = 150
sample_size_4 = 150; n_dim_4 = 200
sample_size_5 = 200; n_dim_5 = 250

## #4(d): $p/n=150/100$

In [15]:
df1, df2 = func_simul(sample_size_3, n_dim_3, n_info, num_rep, step_size, rnd_seed, repro)

  1%|          | 2/200 [00:00<00:17, 11.02it/s]

compute the simulation with sample size 100 and number of variables 150


100%|██████████| 200/200 [00:19<00:00, 10.38it/s]


In [16]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,4.99,5.0,0.0


In [17]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,0.98,0.98,0.98,0.98,0.99


## #4(e): $p/n=200/150$

In [18]:
df1, df2 = func_simul(sample_size_4, n_dim_4, n_info, num_rep, step_size, rnd_seed, repro)

  0%|          | 1/200 [00:00<00:25,  7.93it/s]

compute the simulation with sample size 150 and number of variables 200


100%|██████████| 200/200 [00:24<00:00,  8.18it/s]


In [19]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,5.16,5.0,0.0


In [20]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,1.0,1.0,1.0,1.0,1.0


## #4(f): $p/n=250/200$

In [21]:
df1, df2 = func_simul(sample_size_5, n_dim_5, n_info, num_rep, step_size, rnd_seed, repro)

  0%|          | 1/200 [00:00<00:31,  6.31it/s]

compute the simulation with sample size 200 and number of variables 250


100%|██████████| 200/200 [00:31<00:00,  6.31it/s]


In [22]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,5.13,5.0,0.0


In [23]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,1.0,1.0,1.0,1.0,1.0


# Part 3 : $\log(p)/n \rightarrow 0$

In [24]:
sample_size_6 = 200; n_dim_6 = 400
sample_size_7 = 400; n_dim_7 = 800
sample_size_8 = 600; n_dim_8 = 1200

## #4(g): $p/n=400/200$

In [25]:
df1, df2 = func_simul(sample_size_6, n_dim_6, n_info, num_rep, step_size, rnd_seed, repro)

  0%|          | 0/200 [00:00<?, ?it/s]

compute the simulation with sample size 200 and number of variables 400


100%|██████████| 200/200 [00:38<00:00,  5.19it/s]


In [26]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,5.12,5.0,0.0


In [27]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,1.0,1.0,1.0,1.0,1.0


## #4(h): $p/n=800/400$

In [28]:
df1, df2 = func_simul(sample_size_7, n_dim_7, n_info, num_rep, step_size, rnd_seed, repro)

  0%|          | 0/200 [00:00<?, ?it/s]

compute the simulation with sample size 400 and number of variables 800


100%|██████████| 200/200 [03:02<00:00,  1.09it/s]


In [29]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,5.24,5.0,0.0


In [30]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,1.0,1.0,1.0,1.0,1.0


## #4(i): $p/n=1200/600$

In [31]:
df1, df2 = func_simul(sample_size_8, n_dim_8, n_info, num_rep, step_size, rnd_seed, repro)

  0%|          | 0/200 [00:00<?, ?it/s]

compute the simulation with sample size 600 and number of variables 1200


100%|██████████| 200/200 [05:23<00:00,  1.62s/it]


In [32]:
df1

,algo,Number of selected variables (mean),Number of selected variables (median),Number of selected variables (IQR)
0,solar + hold-out,5.26,5.0,0.0


In [33]:
df2

,algo,Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1),Pr(select X1)
0,solar + hold-out,1.0,1.0,1.0,1.0,1.0


# output the raw results into HTML

In [1]:
!rm -rf Simul_1d.html
!jupyter nbconvert --to html Simul_1d.ipynb 

[NbConvertApp] Converting notebook Simul_1d.ipynb to html
[NbConvertApp] Writing 644052 bytes to Simul_1d.html
